In [3]:
import csv
import datetime
import json
import os

import geopandas as gpd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import requests
from matplotlib.pyplot import connect

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cnbga\AppData\Roaming\nltk_data...


True

In [7]:
sen = SentimentIntensityAnalyzer()
sen1 = sen.polarity_scores('Thank You')
sen2 = sen.polarity_scores('Fuck you')
sen3 = sen.polarity_scores('Shut up and fucking yeah')
print(sen1)
print(sen2)
print(sen3)

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}
{'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'compound': -0.5423}
{'neg': 0.0, 'neu': 0.616, 'pos': 0.384, 'compound': 0.3597}


,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [188]:
df_cases = pd.read_csv('global-data-bycount.csv')
df_tweet = pd.read_csv('data.csv')
df_tweet.set_index('created_at', inplace=True)

print()
tweet_inds = df_tweet.index.tolist()
res_dict = {}
for index in tweet_inds: 
    if index not in res_dict.keys():
        res_dict[index] = []
        alts_for_ind_aday = df_tweet.loc[index, 'attitude']
        for alt in alts_for_ind_aday:
            res_dict[index].append(alt)

for day in res_dict.keys():
    single = res_dict[day]
    neg = 0
    tot = 0
    for sing_at in single:
        if sing_at == 'NON_negative':
            neg += 1
        tot += 1
    perc_neg = neg / tot
    res_dict[day] = perc_neg

lsi = list(res_dict.items())
df_attitude = pd.DataFrame(lsi)
df_attitude.set_index('Date_reported', inplace=True)

df_cases = df_cases[['Date_reported', 'New_cases']]
df_cases = df_cases.groupby(by=['Date_reported']).sum()
df_cases = df_cases.loc['2022-03-03':'2022-03-06']

print(df_cases)
merged = df_cases.join(df_attitude, how='inner')
merged.index.name = 'date'
merged = merged.rename(columns={1:'Non-negative-PERCENT'})
print(merged)



KeyError: "None of ['Date_reported'] are in the columns"